In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
import tensorflow as tf
from tensorflow import keras
import gc
import random
from tensorflow.keras.optimizers import RMSprop
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.layers import BatchNormalization
from keras.layers import BatchNormalization
from sklearn import preprocessing
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from tensorflow.keras.optimizers import RMSprop,Adam,Optimizer,Optimizer
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, MaxPooling2D, BatchNormalization, \
                        Permute, TimeDistributed, Bidirectional, GRU, SimpleRNN, LSTM, GlobalAveragePooling2D, SeparableConv2D
from keras import models
from keras import layers


In [2]:
metadata_file = "C:/Extra project/Skin Cancer/HAM10000_metadata.csv"
cancer = pd.read_csv(metadata_file)

In [3]:
cancer["path"] = ""

In [4]:
part1_folder = "C:/Extra project/Skin Cancer/HAM10000_images_part_1"
part2_folder = "C:/Extra project/Skin Cancer/HAM10000_images_part_2"

cancer.fillna({'age': np.mean(cancer['age'])}, inplace=True)
image_paths = []
for part in ("part_1", "part_2"):
    image_paths += glob.glob("C:/Extra project/Skin Cancer/HAM10000_images_" + part + "/*")
image_ids_n_paths = {os.path.splitext(os.path.basename(path))[0]: path for path in image_paths}
cancer['path'] = cancer['image_id'].map(image_ids_n_paths)
labels = pd.get_dummies(cancer['dx']).iloc[:, :4].values  # Modify to keep only 4 columns

In [5]:
def load_image(path):
    # Implement your image loading and preprocessing logic here
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [90, 120])
    image = image / 255
    return image

In [6]:
train_data, test_val_data = train_test_split(cancer, test_size=0.3, random_state=42)
test_data, val_data = train_test_split(test_val_data, test_size=0.5, random_state=42)

train_images = np.array([load_image(path) for path in train_data['path'].values])
train_labels = pd.get_dummies(train_data['dx']).iloc[:, :4].values  # Modify to keep only 4 columns

test_images = np.array([load_image(path) for path in test_data['path'].values])
test_labels = pd.get_dummies(test_data['dx']).iloc[:, :4].values  # Modify to keep only 4 columns

val_images = np.array([load_image(path) for path in val_data['path'].values])
val_labels = pd.get_dummies(val_data['dx']).iloc[:, :4].values  # Modify to keep only 4 columns

In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(90, 120, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(24, (3, 3), activation="relu", padding="same"))
model.add(Dropout(0.2))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(4, activation="softmax"))
model.summary()
number_of_layers = len(model.layers)
print("Number of layers:", number_of_layers)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 88, 118, 32)       896       
                                                                 
 batch_normalization (BatchN  (None, 88, 118, 32)      128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 44, 59, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 59, 24)        6936      
                                                                 
 dropout (Dropout)           (None, 44, 59, 24)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 22, 29, 24)       0

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min"),
    tf.keras.callbacks.ModelCheckpoint(monitor="val_accuracy", save_best_only=True, filepath="./my_model")
]

In [10]:
history = model.fit(
    train_images, train_labels,
    validation_data=(val_images, val_labels),
    epochs=10, batch_size=32, callbacks=callbacks
)

Epoch 1/10
220/220 [==============================] - 126s 567ms/step - loss: 1458362.1250 - accuracy: 0.2310 - val_loss: 1094410.7500 - val_accuracy: 0.1211
Epoch 2/10
220/220 [==============================] - 194s 882ms/step - loss: 55878448.0000 - accuracy: 0.2351 - val_loss: 30020592.0000 - val_accuracy: 0.1211
Epoch 3/10
220/220 [==============================] - 194s 884ms/step - loss: 405766912.0000 - accuracy: 0.2372 - val_loss: 68249048.0000 - val_accuracy: 0.1211
Epoch 4/10
220/220 [==============================] - 214s 973ms/step - loss: 1581678976.0000 - accuracy: 0.2399 - val_loss: 195208160.0000 - val_accuracy: 0.8184
Epoch 5/10
220/220 [==============================] - 199s 905ms/step - loss: 3339095040.0000 - accuracy: 0.2361 - val_loss: 1460805888.0000 - val_accuracy: 0.0472
Epoch 6/10
220/220 [==============================] - 199s 906ms/step - loss: 5336521216.0000 - accuracy: 0.2341 - val_loss: 2603526400.0000 - val_accuracy: 0.1211


In [15]:
test_generator = tf.keras.preprocessing.image.ImageDataGenerator().flow(test_images, test_labels, batch_size=32)

test_loss, test_accuracy = model.evaluate(test_generator)
print('Test Loss:'+ "%.4f" % test_loss)
print('Test Accuracy:'  + "%.2f" % test_accuracy)

47/47 [==============================] - 13s 223ms/step - loss: 2645356800.0000 - accuracy: 0.1119 9s
Test Loss:2645356800.0000
Test Accuracy:0.11
